In [ ]:
import os
import sys

%matplotlib inline
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches
from operator import add
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import pairwise_distances, silhouette_score, precision_score, recall_score
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx
from sklearn.model_selection import train_test_split
sns.set()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
np.random.seed(42)

In [ ]:
feature = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","target"]

In [ ]:
train_data=pd.read_csv("/content/drive/My Drive/kddcup.data.gz", names = feature)

In [ ]:
train_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,target
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [ ]:
train = np.asarray(train_data)

In [ ]:
cols = train_data.iloc[:,-1:]

In [ ]:
lmapping = {'normal.' : 0 , 'buffer_overflow.' : 1 , 'loadmodule.' : 2 , 'perl.' : 3 , 'neptune.' : 4,
       'smurf.' : 5 , 'guess_passwd.' : 6 , 'pod.' : 7 , 'teardrop.' : 8 , 'portsweep.' : 9 ,
       'ipsweep.' : 17 , 'land.' : 18 , 'ftp_write.' : 19 , 'back.' : 20 , 'imap.' : 21 , 'satan.' : 22 ,
       'phf.' : 10 , 'nmap.' : 11 , 'multihop.' : 12 , 'warezmaster.' : 13 , 'warezclient.' : 14 ,
       'spy.' : 15 , 'rootkit.' : 16 }
cols['target'] = cols['target'].replace(lmapping)

In [ ]:
label = np.asarray(cols)

In [ ]:
vector= train_data.loc[ : , train_data.columns != 'target']

In [ ]:
vector.drop('num_outbound_cmds', axis = 1, inplace = True)

<ipython-input-12-4e9513c62c8b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vector.drop('num_outbound_cmds', axis = 1, inplace = True)


In [ ]:
pmapping = { 'tcp' : 0 , 'http' : 1 , 'SF' : 2 , 'udp' : 3 , 'icmp' : 4 , 'smtp' : 5 , 'finger' : 6 , 'domain_u' : 7, 'auth' : 8, 'telnet' : 9, 'ftp' : 10, 'eco_i' : 11, 'ntp_u' : 12,
       'ecr_i' : 13, 'other' : 14 , 'private' : 15 , 'pop_3' : 16 , 'ftp_data' : 17 , 'rje' : 18 , 'time' : 19,
       'mtp' : 20 , 'link' : 21 , 'remote_job' : 22 , 'gopher' : 23 , 'ssh' : 24 , 'name' : 25 , 'whois' : 26 ,
       'domain' : 27 , 'login' : 28 , 'imap4' : 29 , 'daytime' : 30 , 'ctf' : 31 , 'nntp' : 32 , 'shell' : 33 ,
       'IRC' : 34 , 'nnsp' : 35 , 'http_443' : 36 , 'exec' : 37 , 'printer' : 38 , 'efs' : 39 , 'courier' : 40 ,
       'uucp' : 41 , 'klogin' : 42 , 'kshell' : 43 , 'echo' : 44 , 'discard' : 45 , 'systat' : 46 , 'supdup' : 47 ,
       'iso_tsap' : 48 , 'hostnames' : 49 , 'csnet_ns' : 50 , 'pop_2' : 51 , 'sunrpc' : 52 ,
       'uucp_path' : 53 , 'netbios_ns' : 54 , 'netbios_ssn' : 55 , 'netbios_dgm' : 56 , 'sql_net' : 57 ,
       'vmnet' : 58 , 'bgp' : 59 , 'Z39_50' : 60 , 'ldap' : 61 , 'netstat' : 62 , 'urh_i' : 63 , 'X11' : 64 ,
       'urp_i' : 65 , 'pm_dump' : 66 , 'tftp_u' : 67 , 'tim_i' : 68 , 'red_i' : 69 , 'S1' : 70 , 'REJ' : 71 , 'S2' : 72 , 'S0' : 73 , 'S3' : 74 , 'RSTO' : 75 , 'RSTR' : 76 , 'RSTOS0' : 77 , 'OTH' : 78 ,
       'SH' : 79}
vector['protocol_type'] = vector['protocol_type'].replace(pmapping)
vector['service'] = vector['service'].replace(pmapping)
vector['flag'] = vector['flag'].replace(pmapping)

<ipython-input-13-1f902e99f77a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vector['protocol_type'] = vector['protocol_type'].replace(pmapping)
<ipython-input-13-1f902e99f77a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vector['service'] = vector['service'].replace(pmapping)
<ipython-input-13-1f902e99f77a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [ ]:
X_train, _, y_train, _ = train_test_split(vector, label, train_size = 0.0025, stratify = label , random_state=42)

In [ ]:
df = pd.DataFrame(data=X_train)
df.to_csv('/content/drive/My Drive/train.csv', index=False)

In [ ]:
df = pd.DataFrame(data=y_train)
df.to_csv('/content/drive/My Drive/labels.csv', index=False)

In [ ]:
X_train=pd.read_csv("/content/drive/My Drive/train.csv")
y_train=pd.read_csv("/content/drive/My Drive/labels.csv", names=["label"])

In [ ]:
X_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,0,4,13,2,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0
1,0,4,13,2,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0
2,0,4,13,2,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0
3,0,0,15,71,0,0,0,0,0,0,...,255,2,0.01,0.07,0.0,0.0,0.0,0.0,1.0,1.0
4,0,0,15,73,0,0,0,0,0,0,...,255,9,0.04,0.07,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12241,0,4,13,2,520,0,0,0,0,0,...,255,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0
12242,0,4,13,2,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0
12243,0,0,1,2,302,1380,0,0,0,0,...,255,246,0.96,0.01,0.0,0.0,0.0,0.0,0.0,0.0
12244,0,4,13,2,1032,0,0,0,0,0,...,255,255,1.00,0.00,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y_train

,label
0,0
1,5
2,5
3,5
4,4
...,...
12242,5
12243,5
12244,0
12245,5


In [ ]:
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)
y_train

array([[0],
       [5],
       [5],
       ...,
       [0],
       [5],
       [5]])

In [ ]:
# adjacency matrix
W = cosine_similarity(X_train)
print(W)

[[1.000 1.000 1.000 ... 0.231 1.000 0.945]
 [1.000 1.000 1.000 ... 0.231 1.000 0.945]
 [1.000 1.000 1.000 ... 0.231 1.000 0.945]
 ...
 [0.231 0.231 0.231 ... 1.000 0.231 0.205]
 [1.000 1.000 1.000 ... 0.231 1.000 0.945]
 [0.945 0.945 0.945 ... 0.205 0.945 1.000]]


In [ ]:
# degree matrix
D = np.diag(np.sum(W, axis=1))
print('degree matrix:')
print(D)
# laplacian matrix
L = D - W
print('laplacian matrix:')
print(L)

degree matrix:
[[8840.001 0.000 0.000 ... 0.000 0.000 0.000]
 [0.000 8840.001 0.000 ... 0.000 0.000 0.000]
 [0.000 0.000 8840.001 ... 0.000 0.000 0.000]
 ...
 [0.000 0.000 0.000 ... 3723.549 0.000 0.000]
 [0.000 0.000 0.000 ... 0.000 8840.001 0.000]
 [0.000 0.000 0.000 ... 0.000 0.000 8887.145]]
laplacian matrix:
[[8839.001 -1.000 -1.000 ... -0.231 -1.000 -0.945]
 [-1.000 8839.001 -1.000 ... -0.231 -1.000 -0.945]
 [-1.000 -1.000 8839.001 ... -0.231 -1.000 -0.945]
 ...
 [-0.231 -0.231 -0.231 ... 3722.549 -0.231 -0.205]
 [-1.000 -1.000 -1.000 ... -0.231 8839.001 -0.945]
 [-0.945 -0.945 -0.945 ... -0.205 -0.945 8886.145]]


In [ ]:
e, v = np.linalg.eig(L)

In [ ]:
L

array([[8839.001, -1.000, -1.000, ..., -0.231, -1.000, -0.945],
       [-1.000, 8839.001, -1.000, ..., -0.231, -1.000, -0.945],
       [-1.000, -1.000, 8839.001, ..., -0.231, -1.000, -0.945],
       ...,
       [-0.231, -0.231, -0.231, ..., 3722.549, -0.231, -0.205],
       [-1.000, -1.000, -1.000, ..., -0.231, 8839.001, -0.945],
       [-0.945, -0.945, -0.945, ..., -0.205, -0.945, 8886.145]])

In [ ]:
vec=v.real

In [ ]:
v

array([[ 9.03655451e-03+0.j, -1.53621609e-04+0.j,  1.63333381e-04+0.j,
        ..., -2.97019703e-04+0.j,  2.97019703e-04+0.j,
         2.61551246e-03+0.j],
       [ 9.03655451e-03+0.j, -1.53621609e-04+0.j,  1.63333381e-04+0.j,
        ...,  6.14475188e-03+0.j, -6.14475183e-03+0.j,
         3.21215623e-02+0.j],
       [ 9.03655451e-03+0.j, -1.53621609e-04+0.j,  1.63333381e-04+0.j,
        ..., -1.21036271e-02+0.j,  1.21036269e-02+0.j,
        -3.79482434e-02+0.j],
       ...,
       [ 9.03655451e-03+0.j, -1.54390240e-04+0.j,  1.64333117e-04+0.j,
        ...,  6.61429491e-17+0.j, -6.61429487e-17+0.j,
         3.14863115e-16+0.j],
       [ 9.03655451e-03+0.j, -1.53621609e-04+0.j,  1.63333381e-04+0.j,
        ..., -6.42298444e-05+0.j,  6.42298433e-05+0.j,
        -3.16178111e-05+0.j],
       [ 9.03655451e-03+0.j, -1.53914379e-04+0.j,  1.63665865e-04+0.j,
        ..., -3.97828578e-15+0.j,  3.97828574e-15+0.j,
         6.70956807e-15+0.j]])

In [ ]:
vec

array([[0.009, -0.000, 0.000, ..., -0.000, 0.000, 0.003],
       [0.009, -0.000, 0.000, ..., 0.006, -0.006, 0.032],
       [0.009, -0.000, 0.000, ..., -0.012, 0.012, -0.038],
       ...,
       [0.009, -0.000, 0.000, ..., 0.000, -0.000, 0.000],
       [0.009, -0.000, 0.000, ..., -0.000, 0.000, -0.000],
       [0.009, -0.000, 0.000, ..., -0.000, 0.000, 0.000]])

In [ ]:
norm = np.linalg.norm(vec, axis=0, keepdims=True)
normalized_Y = vec / norm

In [ ]:
np.unique(vec)

array([-1.000, -1.000, -1.000, ..., 1.000, 1.000, 1.000])

In [ ]:
np.savetxt('/content/drive/My Drive/normy.csv', normalized_Y, delimiter=',')

In [ ]:
normalizedY = np.loadtxt('/content/drive/My Drive/normy.csv', delimiter=',')
np.unique(normalizedY)

array([-1.000, -1.000, -1.000, ..., 1.000, 1.000, 1.000])

In [ ]:
kmeans = KMeans(n_clusters=23, random_state=42).fit(normalizedY)

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
predict = kmeans.predict(normalizedY)

In [ ]:
for i in np.unique(predict):
  print(f' count  {i} = {np.count_nonzero(predict == i)}')

 count  0 = 10059
 count  1 = 410
 count  2 = 260
 count  3 = 369
 count  4 = 1
 count  5 = 22
 count  6 = 20
 count  7 = 128
 count  8 = 71
 count  9 = 2
 count  10 = 1
 count  11 = 9
 count  12 = 3
 count  13 = 18
 count  14 = 304
 count  15 = 35
 count  16 = 1
 count  17 = 8
 count  18 = 8
 count  19 = 1
 count  20 = 2
 count  21 = 139
 count  22 = 375


In [ ]:
def mapping(predicted, true, K):
    mape = np.zeros ( (K, 23))
    maped = np.zeros (12246)
    for i in range(12246):
        mape [predicted[i]][true[i]] = mape [predicted[i]][true[i]] + 1
    for i in range(K):
        t = np. argmax (mape , axis=1)
    for i in range (12246) :
        maped[i] = t[predicted[i]]
    return maped

In [ ]:
map=mapping(predict,y_train,23)

In [ ]:
for i in np.unique(map):
  print(f' count  {i} = {np.count_nonzero(map == i)}')

 count  0.0 = 30
 count  4.0 = 3
 count  5.0 = 12213


In [ ]:
np.unique(y_train)

array([ 0,  4,  5,  7,  8,  9, 11, 14, 17, 20, 22])

In [ ]:
map=map.astype(int)

In [ ]:
def conditional_entropy_python(K,predicted,true):
    entropy=0
    rows=len(true)
    mape=np.zeros((K,23))
    labels_number=len(np.unique(y_train))
    for i in range(12246):
        mape [predicted[i]][true[i]] = mape [predicted[i]][true[i]] + 1
    for i in range(K):
        row_entropy=0
        for j in range(labels_number):
            if mape[i][j]!=0:
                fraction_of_label = mape[i][j]/mape[i].sum()
                row_entropy = row_entropy - (fraction_of_label * math.log(fraction_of_label))
        entropy = entropy + (mape[i].sum()/mape.sum())*row_entropy
    return entropy

In [ ]:
cEntropy = conditional_entropy_python(23,map,y_train)
cEntropy

0.98764401124876

In [ ]:
y_train = y_train[1:]

In [ ]:
precision=precision_score(y_train,map, average='micro')
precision

0.5733300669606403

In [ ]:
recall=recall_score(y_train,map,average='micro')
recall

0.43385595296423324

In [ ]:
f1= (2 * precision * recall ) / (precision + recall )
f1

0.4939358919672258

In [ ]:
X=X_train.astype(float)
X

array([[0.000, 4.000, 13.000, ..., 0.000, 0.000, 0.000],
       [0.000, 4.000, 13.000, ..., 0.000, 0.000, 0.000],
       [0.000, 4.000, 13.000, ..., 0.000, 0.000, 0.000],
       ...,
       [0.000, 0.000, 1.000, ..., 0.000, 0.000, 0.000],
       [0.000, 4.000, 13.000, ..., 0.000, 0.000, 0.000],
       [0.000, 4.000, 13.000, ..., 0.000, 0.000, 0.000]])

In [ ]:
class MeanShift:

    def __init__(self, bandwidth, max_iter=300, tol=1e-4):
        '''
        Define a mean-shift algorithm with known kernel bandwidth.
        '''
        self.bandwidth = bandwidth
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X):
        x = np.array(X)
        n, dim = X.shape
        cluster_centers_lst = []
        cluster_labels = np.zeros(n)
        cluster_centers_old = np.zeros(dim)
        # Initialize each sample.
        for t in range(n):
            cluster_centers = x[t]
            # Iterate until convergence.
            i_iter = 0
            while i_iter < self.max_iter:
                if np.linalg.norm(cluster_centers_old - cluster_centers) > self.tol:
                    cluster_centers_old = cluster_centers[:]
                    sum_xi_weight = np.zeros(dim)
                    sum_weight = np.zeros(dim)
                    for i in range(n):
                        r = np.linalg.norm((cluster_centers - x[i]) / self.bandwidth) ** 2
                        weight = 0.5 * np.exp(-0.5 * r)
                        xi_weight = x[i] * weight
                        sum_xi_weight = sum_xi_weight + xi_weight
                        sum_weight = sum_weight + weight
                    cluster_centers = sum_xi_weight / sum_weight
                else:
                    break
                i_iter = i_iter + 1
            cluster_centers_lst.append(np.around(cluster_centers, decimals=2))
        cluster_centers = np.unique(cluster_centers_lst, axis=0)
        # Generate sample labels.
        for i in range(n):
            for j in range(len(cluster_centers)):
                if np.array_equal(cluster_centers_lst[i], cluster_centers[j]):
                    cluster_labels[i] = j
        cluster_labels = cluster_labels.astype(int)
        print(f"\nCluster centers: {cluster_centers}")
        return cluster_labels, cluster_centers

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

In [ ]:
meanshift = MeanShift(bandwidth=2.1, max_iter=50, tol=1e-4)

In [ ]:
cluster_labels, cluster_centers = meanshift.fit(X)

In [ ]:
from sklearn.cluster import MeanShift
clustering = MeanShift(bandwidth=4, max_iter=30).fit(X)

In [ ]:
cluster_centers = clustering.cluster_centers_
cluster_labels = clustering.labels_

In [ ]:
length=len(cluster_centers)

In [ ]:
def mapping(predicted, true, K):
    mape = np.zeros ( (K, 23))
    maped = np.zeros (12246)
    for i in range(12246):
        mape [predicted[i]][true[i]] = mape [predicted[i]][true[i]] + 1
    for i in range(K):
        t = np. argmax (mape , axis=1)
    for i in range (12246) :
        maped[i] = t[predicted[i]]
    return maped, t

In [ ]:
maped, clusters = mapping(cluster_labels,y_train,length)

In [ ]:
len(maped)

12246

In [ ]:
for i in np.unique(maped):
  print(f' count  {i} = {np.count_nonzero(maped == i)}')

 count  0.0 = 2234
 count  4.0 = 2681
 count  5.0 = 7229
 count  7.0 = 1
 count  8.0 = 2
 count  9.0 = 23
 count  11.0 = 4
 count  14.0 = 1
 count  17.0 = 30
 count  20.0 = 4
 count  22.0 = 37


In [ ]:
cluster_labels

array([0, 0, 0, ..., 2, 0, 0])

In [ ]:
cluster_centers

array([[-0.069, 0.797, 0.156, ..., -0.470, -0.244, -0.242],
       [-0.069, -1.265, 0.539, ..., 2.131, -0.245, -0.242],
       [-0.068, -1.006, -1.465, ..., -0.468, -0.233, -0.229],
       ...,
       [-0.069, -1.265, -1.906, ..., -0.470, -0.245, -0.242],
       [-0.069, -1.265, -1.906, ..., -0.470, 2.997, 4.129],
       [-0.069, -1.265, -1.906, ..., -0.470, -0.245, -0.242]])

In [ ]:
np.unique(maped)

array([0.000, 4.000, 5.000, 7.000, 8.000, 9.000, 11.000, 14.000, 17.000,
       20.000, 22.000])

In [ ]:
recall2=recall_score(y_train,maped,average='macro')
recall2

0.8583143518164931

In [ ]:
precision2=precision_score(y_train,maped, average='micro')
precision2

0.9817899722358321

In [ ]:
f12= (2 * precision2 * recall2 ) / (precision2 + recall2 )
f12

0.9159093999450527

In [ ]:
cEntropy2 = conditional_entropy_python(23,maped.astype(int),y_train)


In [ ]:
cEntropy2

0.5299690112487601